# Práctica 10: AEs

En esta práctica usaremos un autoencoder como detector de anomalías. Para ello entrenaremos un autoenconder con un conjunto de imágenes de radiografias de tórax de individuos sanos y consideraremos como anomalías las radiografías de individuos con neumonía

## Dataset

Usaremos un subconjunto de [__MedMNIST__](https://medmnist.com/): un dataset con diversos conjuntos de datos de imágenes médicas. En concreto usaremos __PneumoniaMNIST__, que tiene las siguientes características:
- Más de 5000 imágenes divididas en conjuntos de training, validation y testing.
- Imágenes monocromas de 28x28 píxeles
- Directamente utilizable desde pytorch
- Cada imagen tiene una etiqueta asociada que indica si la imagen es de un individuo sano o con neumonía.

Para poder utilizar el dataset, primero deberemos instalar el paquete medmnist: ``pip install medmnist``

A continuación se incluye un código python que permite un acceso básico al dataset y muestra mosaicos de imágenes de individuos sanos y con neumonía.

In [ ]:
import medmnist
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

print(medmnist.__version__)
from medmnist import PneumoniaMNIST

In [ ]:
# Dataset info
download = True
print(medmnist.INFO['pneumoniamnist'])
DataClass = getattr(__import__('medmnist'), info['python_class'])

# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load data
train_dataset = DataClass(split='train', transform=transform, download=download)

healthy_images = []
pneumonia_images = []

for img, label in train_dataset:
    if label.item() == 0:
        healthy_images.append(img.squeeze())  # remove channel dimension
    if len(healthy_images)==12:
        break

for img, label in train_dataset:
    if label.item() == 1:
        pneumonia_images.append(img.squeeze())  # remove channel dimension
    if len(pneumonia_images)==12:
        break

# Create mosaic
fig, axes = plt.subplots(3, 4, figsize=(5, 4))
fig.suptitle("Healthy (Label = 0) Images from PneumoniaMNIST", fontsize=16)
for i, ax in enumerate(axes.flat):
    ax.imshow(healthy_images[i], cmap='gray')
    ax.axis('off')
plt.tight_layout()


In [ ]:
# Create mosaic
fig, axes = plt.subplots(3, 4, figsize=(5, 4))
fig.suptitle("Non-healthy (Label = 0) Images from PneumoniaMNIST", fontsize=16)
for i, ax in enumerate(axes.flat):
    ax.imshow(pneumonia_images[i], cmap='gray')
    ax.axis('off')
plt.tight_layout()

Obsérvese que el dataset estará compuesto de todas las imágenes, y que para distinguirlas habrá que hacer uso de la etiqueta asociada a cada imagen.

## Modelo

Se probarán las prestaciones de modelos autoencoders estándar (AE) y variacionales (VAE). Como punto de partida se propone la estructura propuesta en el paper disponible en el siguiente [enlace](https://essay.utwente.nl/98159/1/Reve_BA_EEMCS.pdf)

Obsérvese en la fig. 6 del paper la estructura del autoencoder utilizado, en la cual el _bottleneck_ es de tamaño 4x4x8 = 128 sin aplanar

## Tareas a realizar en la práctica

1. Estudiar las características del dataset y plantear un marco experimental adecuado: entrenar con las imágenes del conjunto de training de individuos sanos y usar un conjunto de validación y otro de test balanceados (mismo número de imágenes de individuos sanos y con neumonía)
2. Sobre el conjunto de validación, hacer un estudio de la distribución de los errores (media y desviación de los errores, histograma o nube de puntos como en la fig. 5 del paper) y obtener el umbral que minimiza el error de reconocimiento
3. Con este umbral, calcular las prestaciones del detector de anomalías sobre el conjunto de test. Calcular también probabilidad de falsos positivos y de falsos negativos.